In [ ]:

import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from lightgbm import LGBMClassifier
warnings.filterwarnings('ignore')




In [2]:
data=pd.read_csv("income-classification/income_evaluation.csv")

In [3]:
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
data.isna().sum()

age                0
 workclass         0
 fnlwgt            0
 education         0
 education-num     0
 marital-status    0
 occupation        0
 relationship      0
 race              0
 sex               0
 capital-gain      0
 capital-loss      0
 hours-per-week    0
 native-country    0
 income            0
dtype: int64

In [5]:
data.columns=data.columns.str.strip()

In [6]:
data['income']=[1 if x ==' >50K' else 0 for x in data['income'].values]

In [7]:
data.income.value_counts()/data.shape[0]*100

0    75.919044
1    24.080956
Name: income, dtype: float64

In [8]:
X=data.iloc[:,:-1]
y=data['income']

In [9]:
num_col=[feature for feature in X.columns if X[feature].dtype!='O']
numerical_feature=X[num_col]

In [10]:
numerical_feature.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
0,39,77516,13,2174,0,40
1,50,83311,13,0,0,13
2,38,215646,9,0,0,40
3,53,234721,7,0,0,40
4,28,338409,13,0,0,40


In [11]:
cat_col=[feature for feature in X.columns if X[feature].dtype =="O"]
categorical_feature=X[cat_col]

In [12]:
categorical_feature.head()

,workclass,education,marital-status,occupation,relationship,race,sex,native-country
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba


In [13]:
categorical_feature.workclass=[' Private' if x==' Private' else 'Others' for x in categorical_feature.workclass]

In [14]:
 categorical_feature.workclass.value_counts()

 Private    22696
Others       9865
Name: workclass, dtype: int64

In [15]:
a=categorical_feature.education.value_counts().to_dict()
categorical_feature.education=categorical_feature.education.map(a)

In [16]:
categorical_feature['marital-status'].value_counts()

 Married-civ-spouse       14976
 Never-married            10683
 Divorced                  4443
 Separated                 1025
 Widowed                    993
 Married-spouse-absent      418
 Married-AF-spouse           23
Name: marital-status, dtype: int64

In [17]:
b=categorical_feature['marital-status'].value_counts().to_dict()
categorical_feature['marital-status']=categorical_feature['marital-status'].map(b)

In [18]:
categorical_feature.occupation.value_counts()

 Prof-specialty       4140
 Craft-repair         4099
 Exec-managerial      4066
 Adm-clerical         3770
 Sales                3650
 Other-service        3295
 Machine-op-inspct    2002
 ?                    1843
 Transport-moving     1597
 Handlers-cleaners    1370
 Farming-fishing       994
 Tech-support          928
 Protective-serv       649
 Priv-house-serv       149
 Armed-Forces            9
Name: occupation, dtype: int64

In [19]:
c=categorical_feature.occupation.value_counts().to_dict()
categorical_feature.occupation=categorical_feature.occupation.map(c)

In [20]:
categorical_feature.relationship.value_counts()

 Husband           13193
 Not-in-family      8305
 Own-child          5068
 Unmarried          3446
 Wife               1568
 Other-relative      981
Name: relationship, dtype: int64

In [21]:
d=categorical_feature.relationship.value_counts().to_dict()
categorical_feature.relationship=categorical_feature.relationship.map(d)

In [22]:
categorical_feature.race.value_counts()

 White                 27816
 Black                  3124
 Asian-Pac-Islander     1039
 Amer-Indian-Eskimo      311
 Other                   271
Name: race, dtype: int64

In [23]:
e=categorical_feature.race.value_counts().to_dict()
categorical_feature.race=categorical_feature.race.map(e)

In [24]:
categorical_feature.sex.value_counts()

 Male      21790
 Female    10771
Name: sex, dtype: int64

In [25]:
categorical_feature.sex=categorical_feature.sex.map({' Male':1,' Female':0})

In [26]:
categorical_feature['native-country'].value_counts()

 United-States                 29170
 Mexico                          643
 ?                               583
 Philippines                     198
 Germany                         137
 Canada                          121
 Puerto-Rico                     114
 El-Salvador                     106
 India                           100
 Cuba                             95
 England                          90
 Jamaica                          81
 South                            80
 China                            75
 Italy                            73
 Dominican-Republic               70
 Vietnam                          67
 Guatemala                        64
 Japan                            62
 Poland                           60
 Columbia                         59
 Taiwan                           51
 Haiti                            44
 Iran                             43
 Portugal                         37
 Nicaragua                        34
 Peru                             31
 

In [27]:
categorical_feature['native-country']=[' United-States' if x== ' United-States'else 'Others' for x in categorical_feature['native-country']]

In [28]:
categorical_feature['native-country'].value_counts()

 United-States    29170
Others             3391
Name: native-country, dtype: int64

In [29]:
categorical_feature.head()

,workclass,education,marital-status,occupation,relationship,race,sex,native-country
0,Others,5355,10683,3770,8305,27816,1,United-States
1,Others,5355,14976,4066,13193,27816,1,United-States
2,Private,10501,4443,1370,8305,27816,1,United-States
3,Private,1175,14976,1370,13193,3124,1,United-States
4,Private,5355,14976,4140,1568,3124,0,Others


In [30]:
x=pd.concat([numerical_feature,categorical_feature],axis=1)

In [31]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [32]:
x['native-country']=x['native-country'].astype('category')

In [33]:
x['workclass']=x['workclass'].astype('category')

In [34]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   age             32561 non-null  int64   
 1   fnlwgt          32561 non-null  int64   
 2   education-num   32561 non-null  int64   
 3   capital-gain    32561 non-null  int64   
 4   capital-loss    32561 non-null  int64   
 5   hours-per-week  32561 non-null  int64   
 6   workclass       32561 non-null  category
 7   education       32561 non-null  int64   
 8   marital-status  32561 non-null  int64   
 9   occupation      32561 non-null  int64   
 10  relationship    32561 non-null  int64   
 11  race            32561 non-null  int64   
 12  sex             32561 non-null  int64   
 13  native-country  32561 non-null  category
dtypes: category(2), int64(12)
memory usage: 3.0 MB


In [35]:
lbg=LGBMClassifier()

In [36]:
lbg.fit(X_train,y_train)

ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: workclass, native-country

In [37]:
test_predict=lbg.predict(X_test)
test_accuracy=accuracy_score(y_test,test_predict)
print(" Test Accuracy is {}".format(test_accuracy))

AttributeError: 'NoneType' object has no attribute 'predict'

In [38]:
train_predict=lbg.predict(X_train)
train_accuracy=accuracy_score(y_train,train_predict)
print(" Train Accuracy is {}".format(train_accuracy))

AttributeError: 'NoneType' object has no attribute 'predict'